In [ ]:
import random
import os
import shutil
import glob
import gc
import sys
import h5py
import time
import datetime
import pickle
import librosa
import warnings
import matplotlib.pyplot as plt

import numpy as np
from numpy import array
import pandas as pd
from pandas.plotting import parallel_coordinates
from pydub import AudioSegment

#imported for testing
import wave
import contextlib

# for outputing file
from scipy.cluster.vq import vq, kmeans, whiten
import scipy.stats.stats as st

from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.utils import class_weight

import keras

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical

from colorama import Fore, Back, Style

from IPython.display import clear_output


#warnings.filterwarnings('ignore')
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU
from keras import backend
# print(len(backend.tensorflow_backend._get_available_gpus()) > 0)

#warnings.filterwarnings('ignore')
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

sample_rate = 44100
frame_number = 48
hop_length = 441  # frame size= 2 * hop
segment_length = int(sample_rate * 0.2)  # 0.2
segment_pad = int(sample_rate * 0.02)     # 0.02
overlapping = int(sample_rate * 0.1)   # 0.1

classes = 2
NumofFeaturetoUse = 100
n_neurons = 4096
dense_layers = 1
num_layers = 2
fillength = 10
nbindex = 512
dropout = 0.1
n_batch = 128
n_epoch = 10000

def update_progress(progress):
    bar_length = 100
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

prefix = '..//'
h_feature_vector = np.load(prefix + 'Features//h_feature_vector_48.npy')
h_label_vector = np.load(prefix + 'Features//h_label_vector_48.npy')
a_feature_vector = np.load(prefix + 'Features//a_feature_vector_48.npy')
a_label_vector = np.load(prefix + 'Features//a_label_vector_48.npy')
n_feature_vector = np.load(prefix + 'Features//n_feature_vector_48.npy')
n_label_vector = np.load(prefix + 'Features//n_label_vector_48.npy')
s_feature_vector = np.load(prefix + 'Features//s_feature_vector_48.npy')
s_label_vector = np.load(prefix + 'Features//s_label_vector_48.npy')

h_feature_vector_test = np.load(prefix + 'Features//h_feature_vector_test_48.npy')
h_label_vector_test = np.load(prefix + 'Features//h_label_vector_test_48.npy')
a_feature_vector_test = np.load(prefix + 'Features//a_feature_vector_test_48.npy')
a_label_vector_test = np.load(prefix + 'Features//a_label_vector_test_48.npy')
n_feature_vector_test = np.load(prefix + 'Features//n_feature_vector_test_48.npy')
n_label_vector_test = np.load(prefix + 'Features//n_label_vector_test_48.npy')
s_feature_vector_test = np.load(prefix + 'Features//s_feature_vector_test_48.npy')
s_label_vector_test = np.load(prefix + 'Features//s_label_vector_test_48.npy')

h_label_vector[h_label_vector == 0] = 0
a_label_vector[a_label_vector == 1] = 1
h_label_vector_test[h_label_vector_test == 0] = 0
a_label_vector_test[a_label_vector_test == 1] = 1

h_label_vector = to_categorical(h_label_vector, num_classes = 2)
a_label_vector = to_categorical(a_label_vector, num_classes = 2)
h_label_vector_test = to_categorical(h_label_vector_test, num_classes = 2)
a_label_vector_test = to_categorical(a_label_vector_test, num_classes = 2)

# Load training npy files
featureSet_training = np.vstack((h_feature_vector, a_feature_vector))
label_training = np.vstack((h_label_vector, a_label_vector))

# Load testing npy files
featureSet_testing = np.vstack((h_feature_vector_test, a_feature_vector_test))
label_testing = np.vstack((h_label_vector_test, a_label_vector_test))

def float_compatible(input_np):

    x = np.where(input_np >= np.finfo(np.float32).max)
    for index in range(0, len(x[0])):
        x_position = x[0][index]
        y_position = x[1][index]      
        input_np[x_position, y_position] = 0.0
    input_np = np.nan_to_num(input_np)
        
    return input_np

train_data = float_compatible((featureSet_training).astype(np.float32))
eval_data = float_compatible((featureSet_testing).astype(np.float32))

adam = optimizers.Adam(lr = 3e-6, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0, amsgrad = True)
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
rmsprop = optimizers.RMSprop(lr = 0.0001, rho = 0.9, epsilon = None, decay = 0.0)
adagrad = optimizers.Adagrad(lr = 0.01, epsilon = None, decay = 0.0)
adadelta = optimizers.Adadelta(lr = 1.0, rho = 0.95, epsilon = None, decay = 0.0)
adamax = optimizers.Adamax(lr = 0.002, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0)
nadam = optimizers.Nadam(lr = 0.002, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, schedule_decay = 0.004)

featureSet = train_data
Label = label_training
featureSet = np.split(featureSet, np.array([NumofFeaturetoUse]), axis = 2)[0]

print('training data: ' + str(featureSet.shape))
print('training label: ' + str(Label.shape))

featureSet_val = eval_data
Label_val = label_testing
featureSet_val = np.split(featureSet_val, np.array([NumofFeaturetoUse]), axis = 2)[0]

print('evaluation data: ' + str(featureSet_val.shape))
print('evaluation label: ' + str(Label_val.shape))

def record(str_message, log_file):
    str_message = str_message + '\n'
    file = open(log_file, 'a')
    file.write(str_message)
    file.close()

def create_cnn(title, num_layers, n_neurons, n_batch, nbindex, dropout, classes, dense_layers):

    model = Sequential()

    model.add(Convolution1D(nb_filter=nbindex, filter_length=fillength,
                            input_shape=(featureSet.shape[1], featureSet.shape[2]), kernel_constraint=maxnorm(3)))
    model.add(LeakyReLU(alpha=0.05))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))
    '''
    model.add(Convolution1D(nb_filter=nbindex*2, filter_length=fillength,
                            kernel_constraint=maxnorm(3)))
    model.add(LeakyReLU(alpha=0.05))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))

    model.add(Convolution1D(nb_filter=nbindex*3, filter_length=fillength,
                            kernel_constraint=maxnorm(3)))
    model.add(LeakyReLU(alpha=0.05))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))
    '''
    model.add(Convolution1D(nb_filter=nbindex*5, filter_length=fillength,
                            kernel_constraint=maxnorm(3)))  
    model.add(LeakyReLU(alpha=0.05))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))

    model.add(Flatten())

    model.add(Dense(classes, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

    model.summary()

    return model

def train_cnn():
    
    save_to_path = prefix + str(num_layers) + "_Layer(s)//"

    checkpoint_filepath = prefix + str(num_layers) + "_Layer(s)//Checkpoint_" + title + ".hdf5"
    final_filepath = prefix + str(num_layers) + "_Layer(s)//Final_" + title + ".hdf5"

    if not os.path.exists(save_to_path):
        os.mkdir(save_to_path)

    X, X_test, Y, Y_test= train_test_split(featureSet, Label, test_size = 0.25, shuffle = True)

    model = create_cnn(title, num_layers, n_neurons, n_batch, nbindex, dropout, classes, dense_layers)

    checkpoint = ModelCheckpoint(checkpoint_filepath, monitor = 'val_loss', verbose = 0, save_best_only = True, mode = 'auto')

    early_stopping_monitor = EarlyStopping(patience = 50)

    callbacks_list = [checkpoint, early_stopping_monitor]

    model.fit(X, Y, nb_epoch = n_epoch, batch_size = n_batch,  callbacks = callbacks_list, validation_data = (X_test, Y_test), verbose = 1)

    model.save_weights(final_filepath)

    model.load_weights(checkpoint_filepath)

    return model

def predict_cnn(model):
    y_pred = []
    y_true = []

    for item in list(Label_val):
            if item[0] > item[1]:
                y_true.append(0)
            elif item[0] < item[1]:
                y_true.append(1)
            else:
                y_true.append(0)

    for item in list(model.predict(featureSet_val)):
            if item[0] > item[1]:
                y_pred.append(0)
            elif item[0] < item[1]:
                y_pred.append(1)
            else:
                y_pred.append(0)

    print('Accuracy: ' + str(accuracy_score(y_true, y_pred)))
    print('Precision: ' + str(precision_score(y_true, y_pred)))
    print('Recall: ' + str(recall_score(y_true, y_pred)))
    print('f1 score: ' + str(f1_score(y_true, y_pred)))

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    print('true positive ' + str(tp))
    print('false positive ' + str(fp))
    print('false negative ' + str(fn))
    print('true negative ' + str(tn))

title = 'H_A_neurons_' + str(n_neurons) + '_filters_' + str(
    nbindex) + '_dropout_' + str(dropout) + '_epoch_' + str(n_epoch)

final_filepath = prefix + str(num_layers) + "_Layer(s)//Final_" + title + ".hdf5"
#model = load_model(final_filepath)
model = train_cnn()
predict_cnn(model)


training data: (16632, 48, 100)
training label: (16632, 2)
evaluation data: (3684, 48, 100)
evaluation label: (3684, 2)


C:\Users\Ash Gao\Anaconda3\lib\site-packages\ipykernel_launcher.py:192: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(48, 100), kernel_constraint=<keras.con..., filters=512, kernel_size=10)`
C:\Users\Ash Gao\Anaconda3\lib\site-packages\ipykernel_launcher.py:210: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_constraint=<keras.con..., filters=2560, kernel_size=10)`


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 39, 512)           512512    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 39, 512)           0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 19, 512)           0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 19, 512)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 10, 2560)          13109760  
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 10, 2560)          0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 5, 2560)          

C:\Users\Ash Gao\Anaconda3\lib\site-packages\ipykernel_launcher.py:245: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 12474 samples, validate on 4158 samples
Epoch 1/10000
12474/12474 [==============================] - 347s 28ms/step - loss: 0.6857 - acc: 0.5625 - val_loss: 0.6753 - val_acc: 0.6109
Epoch 2/10000
12474/12474 [==============================] - 341s 27ms/step - loss: 0.6714 - acc: 0.6009 - val_loss: 0.6616 - val_acc: 0.6323
Epoch 3/10000
12474/12474 [==============================] - 329s 26ms/step - loss: 0.6609 - acc: 0.6144 - val_loss: 0.6518 - val_acc: 0.6436
Epoch 4/10000
12474/12474 [==============================] - 330s 26ms/step - loss: 0.6516 - acc: 0.6305 - val_loss: 0.6411 - val_acc: 0.6558
Epoch 5/10000
12474/12474 [==============================] - 333s 27ms/step - loss: 0.6432 - acc: 0.6413 - val_loss: 0.6330 - val_acc: 0.6643
Epoch 6/10000
12474/12474 [==============================] - 335s 27ms/step - loss: 0.6356 - acc: 0.6526 - val_loss: 0.6257 - val_acc: 0.6700
Epoch 7/10000
12474/12474 [==============================] - 336s 27ms/step - loss: 0.6293 - acc: 0

...

12474/12474 [==============================] - 328s 26ms/step - loss: 0.3582 - acc: 0.8629 - val_loss: 0.4499 - val_acc: 0.7888
Epoch 116/10000
12474/12474 [==============================] - 327s 26ms/step - loss: 0.3571 - acc: 0.8611 - val_loss: 0.4503 - val_acc: 0.7888
Epoch 117/10000
12474/12474 [==============================] - 328s 26ms/step - loss: 0.3565 - acc: 0.8628 - val_loss: 0.4498 - val_acc: 0.7886
Epoch 118/10000
12474/12474 [==============================] - 326s 26ms/step - loss: 0.3554 - acc: 0.8618 - val_loss: 0.4505 - val_acc: 0.7896
Epoch 119/10000
12474/12474 [==============================] - 330s 26ms/step - loss: 0.3506 - acc: 0.8679 - val_loss: 0.4473 - val_acc: 0.7905
Epoch 120/10000
12474/12474 [==============================] - 325s 26ms/step - loss: 0.3500 - acc: 0.8676 - val_loss: 0.4459 - val_acc: 0.7900
Epoch 121/10000
12474/12474 [==============================] - 326s 26ms/step - loss: 0.3483 - acc: 0.8697 - val_loss: 0.4483 - val_acc: 0.7908
Epoch 12

12474/12474 [==============================] - 327s 26ms/step - loss: 0.2676 - acc: 0.9133 - val_loss: 0.4114 - val_acc: 0.8110
Epoch 173/10000
12474/12474 [==============================] - 329s 26ms/step - loss: 0.2677 - acc: 0.9097 - val_loss: 0.4101 - val_acc: 0.8071
Epoch 174/10000
12474/12474 [==============================] - 336s 27ms/step - loss: 0.2664 - acc: 0.9118 - val_loss: 0.4091 - val_acc: 0.8081
Epoch 175/10000
12474/12474 [==============================] - 345s 28ms/step - loss: 0.2650 - acc: 0.9155 - val_loss: 0.4085 - val_acc: 0.8090
Epoch 176/10000
12474/12474 [==============================] - 326s 26ms/step - loss: 0.2634 - acc: 0.9129 - val_loss: 0.4131 - val_acc: 0.8093
Epoch 177/10000
12474/12474 [==============================] - 328s 26ms/step - loss: 0.2637 - acc: 0.9145 - val_loss: 0.4068 - val_acc: 0.8114
Epoch 178/10000
12474/12474 [==============================] - 323s 26ms/step - loss: 0.2611 - acc: 0.9139 - val_loss: 0.4094 - val_acc: 0.8127
Epoch 17

12474/12474 [==============================] - 396s 32ms/step - loss: 0.1957 - acc: 0.9480 - val_loss: 0.3786 - val_acc: 0.8264
Epoch 230/10000
12474/12474 [==============================] - 371s 30ms/step - loss: 0.1957 - acc: 0.9457 - val_loss: 0.3773 - val_acc: 0.8290
Epoch 231/10000
12474/12474 [==============================] - 337s 27ms/step - loss: 0.1929 - acc: 0.9460 - val_loss: 0.3790 - val_acc: 0.8283
Epoch 232/10000
12474/12474 [==============================] - 336s 27ms/step - loss: 0.1953 - acc: 0.9459 - val_loss: 0.3782 - val_acc: 0.8314
Epoch 233/10000
12474/12474 [==============================] - 337s 27ms/step - loss: 0.1915 - acc: 0.9468 - val_loss: 0.3798 - val_acc: 0.8276
Epoch 234/10000
12474/12474 [==============================] - 334s 27ms/step - loss: 0.1889 - acc: 0.9488 - val_loss: 0.3763 - val_acc: 0.8309
Epoch 235/10000
12474/12474 [==============================] - 332s 27ms/step - loss: 0.1891 - acc: 0.9491 - val_loss: 0.3758 - val_acc: 0.8300
Epoch 23

12474/12474 [==============================] - 318s 26ms/step - loss: 0.1405 - acc: 0.9688 - val_loss: 0.3587 - val_acc: 0.8398
Epoch 287/10000
12474/12474 [==============================] - 320s 26ms/step - loss: 0.1407 - acc: 0.9697 - val_loss: 0.3591 - val_acc: 0.8425
Epoch 288/10000
12474/12474 [==============================] - 318s 26ms/step - loss: 0.1397 - acc: 0.9695 - val_loss: 0.3577 - val_acc: 0.8418
Epoch 289/10000
12474/12474 [==============================] - 323s 26ms/step - loss: 0.1391 - acc: 0.9697 - val_loss: 0.3569 - val_acc: 0.8442
Epoch 290/10000
12474/12474 [==============================] - 320s 26ms/step - loss: 0.1354 - acc: 0.9719 - val_loss: 0.3577 - val_acc: 0.8444
Epoch 291/10000
12474/12474 [==============================] - 322s 26ms/step - loss: 0.1346 - acc: 0.9710 - val_loss: 0.3567 - val_acc: 0.8430
Epoch 292/10000
12474/12474 [==============================] - 318s 25ms/step - loss: 0.1371 - acc: 0.9698 - val_loss: 0.3567 - val_acc: 0.8451
Epoch 29

12474/12474 [==============================] - 325s 26ms/step - loss: 0.1024 - acc: 0.9843 - val_loss: 0.3473 - val_acc: 0.8502
Epoch 344/10000
12474/12474 [==============================] - 323s 26ms/step - loss: 0.1020 - acc: 0.9829 - val_loss: 0.3440 - val_acc: 0.8528
Epoch 345/10000
12474/12474 [==============================] - 324s 26ms/step - loss: 0.1000 - acc: 0.9853 - val_loss: 0.3444 - val_acc: 0.8511
Epoch 346/10000
12474/12474 [==============================] - 319s 26ms/step - loss: 0.0992 - acc: 0.9848 - val_loss: 0.3442 - val_acc: 0.8514
Epoch 347/10000
12474/12474 [==============================] - 326s 26ms/step - loss: 0.1022 - acc: 0.9804 - val_loss: 0.3437 - val_acc: 0.8523
Epoch 348/10000
12474/12474 [==============================] - 320s 26ms/step - loss: 0.0986 - acc: 0.9844 - val_loss: 0.3447 - val_acc: 0.8523
Epoch 349/10000
12474/12474 [==============================] - 316s 25ms/step - loss: 0.0969 - acc: 0.9837 - val_loss: 0.3558 - val_acc: 0.8413
Epoch 35